<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li></ul></li></ul></div>

In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')
print(nb_name)

<IPython.core.display.Javascript object>

In [2]:
import numpy as np 
import pandas as pd
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

from clustering_utils import *
from eda_utils import *
from myutils_V6 import *



In [3]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=None)
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report
        
- Time complexity 
    - O(n^(dk+1)) where n is the number of observatons, d is the dimensionality, and k is k clusters

## modeling

In [4]:
# dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=2)
# dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=200)
# dtm_test = transform_mapper.transform(dtm_test)

# print('dtm_train.shape', dtm_train.shape)
# print('dtm_test.shape', dtm_test.shape)
# clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=50, metric='Cosine', repeats=2)
# pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
# print(metrics.classification_report(y_true = test_label, y_pred=pred))

In [5]:
dtm_train, dtm_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, min_df=2, max_df=0.99)
dtm_train, transform_mapper = dimension_reduction(dtm_train, out_dim=500)
dtm_test = transform_mapper.transform(dtm_test)

print('dtm_train.shape', dtm_train.shape)
print('dtm_test.shape', dtm_test.shape)
clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=50, metric='L2', repeats=20)
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
print(metrics.classification_report(y_true = test_label, y_pred=pred))

num of words: 55535
Dimension reduction with truncate SVD:
   input columns with  55535
   output columns with  500
dtm_train.shape (11083, 500)
dtm_test.shape (7761, 500)
Group to label mapping: 
Group 0 <-> label comp.windows.x
Group 1 <-> label talk.politics.mideast
Group 2 <-> label comp.windows.x
Group 3 <-> label talk.politics.misc
Group 4 <-> label sci.crypt
Group 5 <-> label talk.politics.guns
Group 6 <-> label comp.windows.x
Group 7 <-> label sci.med
Group 8 <-> label sci.space
Group 9 <-> label talk.religion.misc
Group 10 <-> label comp.graphics
Group 11 <-> label talk.politics.guns
Group 12 <-> label sci.space
Group 13 <-> label misc.forsale
Group 14 <-> label rec.sport.baseball
Group 15 <-> label comp.sys.mac.hardware
Group 16 <-> label sci.crypt
Group 17 <-> label comp.sys.ibm.pc.hardware
Group 18 <-> label sci.med
Group 19 <-> label sci.electronics
Group 20 <-> label sci.med
Group 21 <-> label sci.space
Group 22 <-> label rec.autos
Group 23 <-> label comp.graphics
Group 2

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)